In [13]:
from datetime import datetime as dt

import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor as dtr
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.ensemble import GradientBoostingRegressor as gbr

from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

import xgboost as xgb
from sklearn.model_selection import train_test_split

import tensorflow as tf

url = 'https://drive.google.com/file/d/1-4YpXkd2kIOM5viSRw8g7oOQm8sicciB/view?usp=sharing'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
y_train = pd.read_csv(url, index_col=0)

url = 'https://drive.google.com/file/d/1-7VK3dNry2-AYnfRsxMWsOKhHHMTN_ZA/view?usp=sharing'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
test_indices = pd.read_csv(url, index_col=0)

2022-08-11 02:14:58.071169: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-11 02:14:58.071208: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [14]:
def month_to_season(month: int):
    if month < 4:
        return "winter"
    elif month < 7:
        return "spring"
    elif month < 10:
        return "summer"
    elif month < 13:
        return "fall"

In [15]:
df = pd.read_csv("weather.csv")

In [16]:
df["timestamp"] = df[["timestamp"]].apply(lambda x: f"{x[0][0:4]}-{x[0][4:6]}-{x[0][6:8]}", axis=1)

In [17]:
# df[["timestamp"]].apply(lambda x: f"{x[0][0:4]}-{x[0][4:6]}-{x[0][6:8]}", axis=1)
df = df.groupby("timestamp").mean()
df = df[df.index >= "2009-12-28"]

In [18]:
fitted_df = pd.DataFrame(df.iloc[0:7].sum() / 7, columns=[f"{df.index[0]}/{df.index[6]}"]).transpose()
for i in range(7, len(df.index), 7):
    fitted_df = pd.concat([fitted_df, pd.DataFrame(df.iloc[i:i + 7].sum() / 7, columns=[f"{df.index[i]}/{df.index[i + 6]}"]).transpose()])

In [19]:
fitted_df.index.name="year_weeks"

In [20]:
y_train = pd.concat([y_train, fitted_df.iloc[0:len(y_train.index)]], axis=1)
y_train.index.name = "year_weeks"

In [21]:
train = y_train.reset_index()
melted_train = pd.melt(train, id_vars=["year_weeks", "Temperature", "Relative Humidity", "Wind Speed", "Wind Direction"], var_name="City", value_name="Weight")
melted_train["temp"] = melted_train[["City"]].apply(lambda x: x[0].split("_"), axis=1)
melted_train[["food", "district"]] = pd.DataFrame(melted_train["temp"].tolist())

melted_train["month"] = list(map(lambda x: int(x.split("/")[1].split("-")[1]), melted_train.year_weeks))
melted_train["day"] = list(map(lambda x: int(x.split("/")[1].split("-")[2]), melted_train.year_weeks))
melted_train["season"] = list(map(lambda x: month_to_season(x), melted_train.month))

melted_train = melted_train[["year_weeks", "month", "day", "season", "year_weeks", "Temperature",
                             "Relative Humidity", "Wind Speed", "Wind Direction", "food", "district", "Weight"]]

X_train_df = pd.concat([pd.get_dummies(melted_train["month"]),
                        # pd.get_dummies(melted_train["day"]),
                        # pd.get_dummies(melted_train["season"]),
                        # melted_train["Temperature"],
                        # melted_train["Relative Humidity"],
                        # melted_train["Wind Speed"],
                        # melted_train["Wind Direction"],
                        pd.get_dummies(melted_train["food"]),
                        pd.get_dummies(melted_train["district"])], axis=1)

X_train = X_train_df.to_numpy()
Y_train = melted_train["Weight"].to_numpy()


In [22]:
ddd = pd.DataFrame(y_train.sum(), columns=["sum"])
zeros = ddd[ddd["sum"] == 0].index.tolist()

In [ ]:
X_train_08, X_test_02, Y_train_08, Y_test_02 = train_test_split(X_train, Y_train, test_size=0.2, random_state=0)

model = tf.keras.models.Sequential([
    # tf.keras.layers.Flatten(input_shape=(46, 1))
    tf.keras.layers.Dense(128, input_shape=(46,), activation="relu"),
    # tf.keras.layers.Dense(128, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(500001 , activation='softmax')
])

print(model.summary())

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=5)
# model.evaluate(X_test_02, Y_test_02)


Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_61 (Dense)            (None, 128)               6016      
                                                                 
 dense_62 (Dense)            (None, 500001)            64500129  
                                                                 
Total params: 64,506,145
Trainable params: 64,506,145
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
 730/3079 [======>.......................] - ETA: 19:13 - loss: 2.8727 - accuracy: 0.8643